# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [1]:
# 1. Registra gli ordini in una lista di classi Order



GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

def is_a_number(list):
    for i in range(len(list)):
        if list[i].isdigit():
            return i

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste

        self.quantity = int(lines[1].split(" ")[is_a_number(lines[1].split(" "))])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[5]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[3].split(" ")[-1].replace(lines[3].split(" ")[-1][-1], ''))
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        return ask_price*(self.quantity)
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        return self.Payment(ask_price)-6*(self.quantity)-cost_per_km*(self.Distance( GCS_Piazza_Leo))
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    file=open(f'{i+1}.txt', 'r')
    fragment=file.read()
    orders.append(Order(fragment))
    file.close()
    


In [24]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = [sum([order.Profit(orders[i].bid_price, 0) for order in orders if orders[i].bid_price<order.bid_price]) for i in range(len(orders))]


# Determina il profitto maggiore
max_profit = max(possible_profits)
print(max_profit)

# Determina il prezzo migliore
best_price = orders[possible_profits.index(max_profit)].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if  best_price<ord1.bid_price]
print("Si eseguiranno", len(orders_to_go),"ordini")


358094.13999999996
Il miglior prezzo è €13.03
Si eseguiranno 198 ordini


In [31]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


def costo_trasporto_per_km(n_magliette):
    cost_less_100 = 0.10
    cost_at_least_100 = 0.05
    if n_magliette>=100:
        return cost_at_least_100
    else:
        return cost_less_100
    
# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits =possible_profits = [sum([order.Profit(orders[i].bid_price, costo_trasporto_per_km(order.quantity) ) for order in orders if orders[i].bid_price<order.bid_price]) for i in range(len(orders))]
 

    # Determina il profitto maggiore
max_profit = max(possible_profits)
print(max_profit)

# Determina il prezzo migliore
best_price = orders[possible_profits.index(max_profit)].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go =[ord for ord in orders if best_price<ord.bid_price]
print("Si eseguiranno", len(orders_to_go),"ordini")


def bubbleSort(arr): 
    n = len(arr) 
    for i in range(n-1): 
        for j in range(0, n-i-1): 
            if arr[j].Profit(best_price, costo_trasporto_per_km(arr[j].quantity)) > arr[j+1].Profit(best_price, costo_trasporto_per_km(arr[j+1].quantity)) : 
                arr[j], arr[j+1] = arr[j+1], arr[j] 
    return arr



orders_to_go=bubbleSort(orders_to_go)
orders_to_go.reverse()


_ = [orders_to_go[i].Details() for i in range(len(orders_to_go))]

356448.4370822799
Il miglior prezzo è €13.03
Si eseguiranno 198 ordini
JP Iacovelli Azienda Agricola ha ordinato 497 magliette taglia S a 17.92€ all'indirizzo [45.058065, 6.422715]
Francesco Sosta IB ha ordinato 494 magliette taglia L a 15.6€ all'indirizzo [44.958786, 10.107809]
Manuel Carrara Franchising ha ordinato 492 magliette taglia XS a 19.85€ all'indirizzo [44.811402, 11.070849]
Maurizio Cerisola Frumento ha ordinato 492 magliette taglia S a 16.34€ all'indirizzo [45.149412, 12.169453]
Gianmarco Bressan Company ha ordinato 491 magliette taglia XS a 19.79€ all'indirizzo [45.661732, 10.428504]
JP Carrara Servizi Finanziari ha ordinato 486 magliette taglia XS a 19.87€ all'indirizzo [46.177794, 8.410633]
Giulia Calcaterra Tentazioni ha ordinato 478 magliette taglia S a 13.48€ all'indirizzo [44.425229, 11.301004]
Carlo Augusto Carrara Grissini ha ordinato 478 magliette taglia S a 14.22€ all'indirizzo [45.482738, 12.006968]
Davide Zacchei SPA ha ordinato 477 magliette taglia XS a 17.66

Angelica Bacchiocchi Strumenti Musicali ha ordinato 95 magliette taglia M a 13.87€ all'indirizzo [45.794128, 8.917028]
Francesco Fedeli Frumento ha ordinato 96 magliette taglia L a 17.44€ all'indirizzo [46.547132, 6.943494]
Luca Bacchiocchi Servizi Finanziari ha ordinato 94 magliette taglia S a 16.13€ all'indirizzo [44.598262, 7.289589]
Mirko Iacovelli e Sorelle ha ordinato 89 magliette taglia S a 18.99€ all'indirizzo [44.94402, 9.421879]
Filippo Moneta INC. ha ordinato 86 magliette taglia XL a 16.45€ all'indirizzo [45.652833, 9.288692]
Alessandro Carrara e Fratelli ha ordinato 82 magliette taglia XXL a 15.57€ all'indirizzo [46.472705, 7.279097]
Carlo Augusto Sosta Mascherine ha ordinato 74 magliette taglia XXL a 15.89€ all'indirizzo [45.481015, 9.410362]
Mirko Mescolini SRL ha ordinato 75 magliette taglia XXL a 15.03€ all'indirizzo [46.176933, 7.823514]
Carlo Augusto Sgarbossa Franchising ha ordinato 72 magliette taglia XXL a 13.5€ all'indirizzo [45.248003, 9.682098]
Filippo Vitellio 